# Product categorization

Objectif: Regrouper des items en différentes catégories plus générales en se basant sur une analyse sémantique du contenu de la description.
Contrainte: La description ne fait que quelques mots, c'est une entête.
Un exemple d'application pourrait être un site d'échange/ventes entre particulier où le produit est décrit en qq mots par l'utilisateur. Ou alors des messages dont on n'aurait que le sujet, pas le corps du texte.

Le problème s'identifie à du topic modeling mais avec un texte (document) très court

- Exploration / formatage
- Approche Bag of words suivie d'un k-Means
- A pproche Latent Dirichlet Allocation
- Approches hybrides: guided LDA? Autres?

In [ ]:
data = 